In [18]:
import glob
import os
import pandas as pd
import random
from dotenv import load_dotenv

import dspy
from dspy.evaluate import Evaluate
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

In [2]:
os.environ["DSP_NOTEBOOK_CACHEDIR"] = os.path.join('.', 'cache')

In [19]:
load_dotenv()

True

## Initialize Models

In [3]:
turbo = dspy.OpenAI(model='gpt-3.5-turbo-1106', max_tokens=250, model_type='chat')

dspy.settings.configure(lm=turbo)

In [4]:
gpt4T = dspy.OpenAI(model='gpt-4-1106-preview', max_tokens=350, model_type='chat')

In [22]:
RUN_FROM_SCRATCH = True

# ScoNe Data

In [5]:
!git clone https://github.com/selenashe/ScoNe.git

Cloning into 'ScoNe'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.s:  51% (40/77)
remote: Compressing objects: 100% (55/55), done.
remote: Total 77 (delta 42), reused 42 (delta 20), pack-reused 0
Receiving objects: 100% (77/77), 116.25 KiB | 1.25 MiB/s, done.
Resolving deltas: 100% (42/42), done.


In [6]:
def load_scone(dirname):
    dfs = []
    for filename in glob.glob(dirname + "/*.csv"):
        df = pd.read_csv(filename, index_col=0)
        df['category'] = os.path.basename(filename).replace(".csv", "")
        dfs.append(df)
    data_df = pd.concat(dfs)

    def as_example(row):
        # The 'one_scoped' file is from an earlier dataset, MoNLI, and
        # so is formatted a bit differently:
        suffix = '' if row['category'] == 'one_scoped' else '_edited'
        # Reformat the hypothesis to be an embedded clause in a question:
        hkey = 'sentence2' + suffix
        question = row[hkey][0].lower() + row[hkey][1: ].strip(".")
        question = f"Can we logically conclude for sure that {question}?"
        # Binary task formulation:
        label = "Yes" if row['gold_label' + suffix] == 'entailment' else "No"
        return dspy.Example({
            "context": row['sentence1' + suffix],
            "question": question,
            "answer": label,
            "category": row['category']
        }).with_inputs("context", "question")

    return list(data_df.apply(as_example, axis=1).values)

## Train, Dev, Test

In [7]:
all_train = load_scone("ScoNe/scone_nli/train")

random.seed(1)
random.shuffle(all_train)

# 200 random train, 50 random dev:
train, dev = all_train[: 200], all_train[200: 250]

len(train), len(dev)

(200, 50)

In [34]:
dev

[Example({'context': 'The people are not happy when they play instruments.', 'question': 'Can we logically conclude for sure that the people are not happy when they play accordions?', 'answer': 'No', 'category': 'one_not_scoped'}) (input_keys={'question', 'context'}),
 Example({'context': 'the boy does not play a guitar', 'question': 'Can we logically conclude for sure that the boy does not play an instrument?', 'answer': 'No', 'category': 'one_scoped'}) (input_keys={'question', 'context'}),
 Example({'context': 'The three children not holding plants are not in school today.', 'question': 'Can we logically conclude for sure that the three children not holding poppies are not in school today?', 'answer': 'Yes', 'category': 'one_scoped_one_not_scoped'}) (input_keys={'question', 'context'}),
 Example({'context': 'A woman did not like cheeseburger and did not like clothes.', 'question': 'Can we logically conclude for sure that a woman did not like food and did not like clothes?', 'answer':

In [8]:
random.seed(1)

test = load_scone(dirname=f"ScoNe/scone_nli/test")

# We're developing a system for the full ScoNe benchmark, but we'll
# evaluate only on one of the hardest and most informative ScoNe
# categories for now -- examples with a single negation that plays
# a crucial role in the reasoning:
test = [ex for ex in test if ex.category == "one_scoped"]

In [9]:
pd.Series([ex.answer for ex in test]).value_counts()

No     100
Yes    100
Name: count, dtype: int64

# Evaluation

In [10]:
scone_accuracy = dspy.evaluate.metrics.answer_exact_match

In [11]:
evaluator = Evaluate(devset=test, num_threads=1, display_progress=True, display_table=0)

# Zero Shot Chain of Thought

In [12]:
class ScoNeSignature(dspy.Signature):
    ("""You are given some context (a premise) and a question (a hypothesis). """
    """You must indicate with Yes/No answer whether we can logically """
    """conclude the hypothesis from the premise.""")

    context = dspy.InputField()
    question = dspy.InputField()
    answer = dspy.OutputField(desc="Yes or No")

In [13]:
class ScoNeCoT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_answer = dspy.ChainOfThought(ScoNeSignature)

    def forward(self, context, question):
        return self.generate_answer(context=context, question=question)

In [14]:
cot_zeroshot = ScoNeCoT()

In [20]:
evaluator(cot_zeroshot, metric=scone_accuracy)


Average Metric: 99 / 200  (49.5): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [03:37<00:00,  1.09s/it]

Average Metric: 99 / 200  (49.5%)



/Users/richy/.pyenv/versions/3.11.7/envs/llm/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:187: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)


49.5

# Bootstrap Few Shot Optimization with Random Search

Use "teacher," in this case GPT4, to generate "demonstrations". Demonstrations basically include the "thoughts" or other LLM responses alongside the final answer. These thoughts help guide the LLM to do something similar in the test scenario. Then, uses those examples to do few shot prompting. Examples of this at the end. 

In [21]:
bootstrap_optimizer = BootstrapFewShotWithRandomSearch(
    max_bootstrapped_demos=8,
    max_labeled_demos=8,
    num_candidate_programs=10,
    num_threads=8,
    metric=scone_accuracy,
    teacher_settings=dict(lm=gpt4T))

Going to sample between 1 and 8 traces per predictor.
Will attempt to train 10 candidate sets.


In [23]:
if RUN_FROM_SCRATCH:
    cot_fewshot = bootstrap_optimizer.compile(cot_zeroshot, trainset=train, valset=dev)
else:
    cot_fewshot = ScoNeCoT()
    cot_fewshot.load("scone-cot_fewshot-turbo-gpt4-demos.json")


Average Metric: 24 / 50  (48.0): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:08<00:00,  6.13it/s]


Average Metric: 24 / 50  (48.0%)
Score: 48.0 for set: [0]
New best score: 48.0 for seed -3
Scores so far: [48.0]
Best score: 48.0



Average Metric: 29 / 50  (58.0): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:05<00:00,  8.90it/s]


Average Metric: 29 / 50  (58.0%)
Score: 58.0 for set: [8]
New best score: 58.0 for seed -2
Scores so far: [48.0, 58.0]
Best score: 58.0



  6%|███████▊                                                                                                                                     | 11/200 [00:35<10:17,  3.27s/it]


...

Backing off 0.6 seconds after 1 tries calling function <function GPT3.request at 0x117742200> with kwargs {}
Backing off 0.2 seconds after 1 tries calling function <function GPT3.request at 0x117742200> with kwargs {}



Average Metric: 24 / 46  (52.2):  92%|████████████████████████████████████████████████████████████████████████████████████████████████████▎        | 46/50 [00:07<00:00,  4.72it/s]

Backing off 1.0 seconds after 2 tries calling function <function GPT3.request at 0x117742200> with kwargs {}



Average Metric: 27 / 50  (54.0): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:10<00:00,  4.77it/s]

Average Metric: 27 / 50  (54.0%)
Score: 54.0 for set: [8]
Scores so far: [48.0, 58.0, 60.0, 72.0, 56.0, 70.0, 42.0, 56.0, 66.0, 46.0, 64.0, 50.0, 54.0]
Best score: 72.0
Average of max per entry across top 1 scores: 0.72
Average of max per entry across top 2 scores: 0.94
Average of max per entry across top 3 scores: 0.98
Average of max per entry across top 5 scores: 1.0
Average of max per entry across top 8 scores: 1.0
Average of max per entry across top 9999 scores: 1.0
13 candidate programs found.


In [24]:
evaluator(cot_fewshot, metric=scone_accuracy)


Average Metric: 173 / 200  (86.5): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [03:22<00:00,  1.01s/it]

Average Metric: 173 / 200  (86.5%)


86.5

In [25]:
cot_fewshot.save("scone-cot_fewshot-turbo-gpt4-demos.json")

# Examples

In [26]:
turbo.inspect_history(n=1)





You are given some context (a premise) and a question (a hypothesis). You must indicate with Yes/No answer whether we can logically conclude the hypothesis from the premise.

---

Follow the following format.

Context: ${context}

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: Yes or No

---

Context: It is not true that there is not a single person walking in the city.

Question: Can we logically conclude for sure that it is not true that there is not a single celebrity walking in the city?

Reasoning: Let's think step by step in order to produce the answer. We know that the statement negates the absence of any person walking in the city, which means there is at least one person walking in the city. However, we do not have information about the status of the person or persons walking in the city. They could be celebrities or they could be non-celebrities. Without specific information about the individuals' celebrity st